In [16]:
from openai import AzureOpenAI
client = AzureOpenAI(
 azure_endpoint = openai_api_url,
 api_key= openai_api_key,
 api_version="2024-02-01"
)

In [17]:
deployment_1 = "gpt-35-turbo"

In [40]:
system_message = """You are multi-language translation assistant. A user will provide you with a text to translate between triple pluses (+++) and context around the translation between triple ampersands (&&&). You will translate the text into Spanish, French, German, and Japanese. Once you do that, you will output the translation in JSON format. To accomplish this task, first read the translation context, then translate the text into Spanish, French, German, and Japanese in that order, then output the following JSON format:
{
 "Spanish": "",
 "French": "",
 "German": "",
 "Japanese": ""
}
Only output the JSON and ensure that it is the correct format."""

user_message = """+++Henry, Tomorrow I will be late for work due to unforeseen circumstances. My apologies, Mark.+++ &&&This is an e-mail from a worker to his boss.&&&"""

In [41]:
messages = [
 {"role": "system", "content": system_message},
 {"role": "user", "content": user_message},
 ]

In [42]:
results = client.chat.completions.create(
 model=deployment_1,
 messages=messages)

In [43]:
results_text = results.choices[0].message.content
results_text

'{\n "Spanish": "+++Henry, Mañana llegaré tarde al trabajo debido a circunstancias imprevistas. Mis disculpas, Mark.+++",\n "French": "+++Henry, Demain je serai en retard pour le travail en raison de circonstances imprévues. Mes excuses, Mark.+++",\n "German": "+++Henry, Morgen werde ich aufgrund unvorhergesehener Umstände zu spät zur Arbeit kommen. Meine Entschuldigung, Mark.+++",\n "Japanese": "+++ヘンリーさん、明日は予期せぬ事態が発生し、遅刻してしまいます。申し訳ございません、マーク+++"\n}'

In [44]:
import tiktoken 

# Function to get token count for Open AI
def num_tokens_from_string(string: str) -> int:
 """Returns the number of tokens in a text string."""
 enc = tiktoken.encoding_for_model("gpt-4")
 num_tokens = len(enc.encode(string))
 return num_tokens

num_tokens_from_string(system_message + user_message)

170

In [48]:
attempts = 0
max_attempts = 3
success = False
while not success and attempts < max_attempts:
    try:
        results = client.chat.completions.create(
        model= deployment_1,
        messages=messages
        )
        success = True
    except TimeoutError as e:
        print("Timeout error occurred, trying again...")
        attempts += 1
        time.sleep(5)
if not success:
    print("Failed to get a response after several retries.")

In [52]:
import json 

results_json = json.loads(results.choices[0].message.content)
results_json["Japanese"]


'+++ヘンリー、明日は予期せぬ事情により遅刻することになります。申し訳ありません、マーク。+++'